In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv

--2025-11-02 11:50:51--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 874188 (854K) [text/plain]
Saving to: ‘car_fuel_efficiency.csv.1’

car_fuel_efficiency 100%[===================>] 853.70K  --.-KB/s    in 0.09s   

2025-11-02 11:50:51 (9.01 MB/s) - ‘car_fuel_efficiency.csv.1’ saved [874188/874188]



In [16]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

In [3]:
data = pd.read_csv("car_fuel_efficiency.csv")
data.head()

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369


In [7]:
data.isna().sum()

engine_displacement      0
num_cylinders          482
horsepower             708
vehicle_weight           0
acceleration           930
model_year               0
origin                   0
fuel_type                0
drivetrain               0
num_doors              502
fuel_efficiency_mpg      0
dtype: int64

In [8]:
data.num_cylinders = data.num_cylinders.fillna(0)
data.horsepower = data.horsepower.fillna(0)
data.acceleration = data.acceleration.fillna(0)
data.num_doors = data.num_doors.fillna(0)

In [9]:
data.isna().sum()

engine_displacement    0
num_cylinders          0
horsepower             0
vehicle_weight         0
acceleration           0
model_year             0
origin                 0
fuel_type              0
drivetrain             0
num_doors              0
fuel_efficiency_mpg    0
dtype: int64

In [ ]:
fuel_efficiency_mpg = data.fuel_efficiency_mpg
data = data.drop(columns=["fuel_efficiency_mpg"])

data.head()


,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0
2,170,0.0,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0
3,220,4.0,0.0,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0


In [15]:
data_train, data_other, fuel_efficiency_mpg_train, fuel_efficiency_mpg_other = train_test_split(data, fuel_efficiency_mpg, train_size=0.6, random_state=1)
data_test, data_val, fuel_efficiency_mpg_test, fuel_efficiency_mpg_val = train_test_split(data_other, fuel_efficiency_mpg_other, train_size=0.5, random_state=1)

len(data_train), len(data_test), len(fuel_efficiency_mpg_train), len(fuel_efficiency_mpg_test)

(5822, 1941, 5822, 1941)

In [18]:
dv = DictVectorizer(sparse=False)

dicts = data_train.to_dict(orient="records")
X_train = dv.fit_transform(dicts)
y_train = fuel_efficiency_mpg_train.values

X_train

array([[1.00000000e+01, 0.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 2.51041090e+03],
       [1.27000000e+01, 0.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 3.59748366e+03],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 2.21734962e+03],
       ...,
       [1.65000000e+01, 0.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 2.59665719e+03],
       [1.22000000e+01, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.86038802e+03],
       [1.46000000e+01, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 3.40135769e+03]], shape=(5822, 14))

In [19]:
from sklearn.tree import DecisionTreeRegressor

In [23]:
model = DecisionTreeRegressor(max_depth=1)
model.fit(X_train, y_train)

,criterion,'squared_error'
,splitter,'best'
,max_depth,1
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [29]:
from sklearn.tree import export_text

feature_names = dv.get_feature_names_out()  # or dv.feature_names_ for older sklearn
print(export_text(model, feature_names=list(feature_names)))

|--- vehicle_weight <= 3028.82
|   |--- value: [16.86]
|--- vehicle_weight >  3028.82
|   |--- value: [12.87]



In [35]:
dicts_val = data_val.to_dict(orient="records")

X_val = dv.transform(dicts_val)
y_val = fuel_efficiency_mpg_val.values

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

In [49]:
max_depths = [10, 15, 20, 25]

for md in max_depths:
    rmse = []
    for n_est in range(10, 210, 10):
        model_rf = RandomForestRegressor(n_estimators=n_est, random_state=1, n_jobs=-1, max_depth=md)
        model_rf.fit(X_train, y_train)
        
        r = root_mean_squared_error(y_val, model_rf.predict(X_val))
        rmse.append(r)
    
    print(md, np.mean(rmse))

10 0.43243409050680903
15 0.4349784038599617
20 0.43503510280881075
25 0.43579708794242444


In [53]:
model_q5 = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1, max_depth=20)
model_q5.fit(X_train, y_train)

,n_estimators,10
,criterion,'squared_error'
,max_depth,20
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [54]:
list(zip(feature_names, model_q5.feature_importances_))

[('acceleration', np.float64(0.011468099095561263)),
 ('drivetrain=All-wheel drive', np.float64(0.0003600224940845473)),
 ('drivetrain=Front-wheel drive', np.float64(0.00027847763241198905)),
 ('engine_displacement', np.float64(0.00318029617947635)),
 ('fuel_type=Diesel', np.float64(0.00033056910017469353)),
 ('fuel_type=Gasoline', np.float64(0.0003206886782936228)),
 ('horsepower', np.float64(0.015922368707906254)),
 ('model_year', np.float64(0.0030941279217261163)),
 ('num_cylinders', np.float64(0.002298574646630631)),
 ('num_doors', np.float64(0.0015809895491610006)),
 ('origin=Asia', np.float64(0.00043661577257724513)),
 ('origin=Europe', np.float64(0.0004179682492378307)),
 ('origin=USA', np.float64(0.0004728105311833485)),
 ('vehicle_weight', np.float64(0.9598383914415751))]

In [58]:
import xgboost as xgb
list(feature_names)

['acceleration',
 'drivetrain=All-wheel drive',
 'drivetrain=Front-wheel drive',
 'engine_displacement',
 'fuel_type=Diesel',
 'fuel_type=Gasoline',
 'horsepower',
 'model_year',
 'num_cylinders',
 'num_doors',
 'origin=Asia',
 'origin=Europe',
 'origin=USA',
 'vehicle_weight']

In [59]:
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=list(feature_names))
dval= xgb.DMatrix(X_val, label=y_val, feature_names=list(feature_names))

In [65]:
xgb_params = {
    'eta': 0.1, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

In [66]:
watchlist = [(dtrain, "train"), (dval, "val")]

In [67]:
xg_model = xgb.train(xgb_params, dtrain, num_boost_round=100, evals=watchlist)

[0]	train-rmse:2.31334	val-rmse:2.32561
[1]	train-rmse:2.09552	val-rmse:2.10973
[2]	train-rmse:1.90001	val-rmse:1.91639
[3]	train-rmse:1.72438	val-rmse:1.74254
[4]	train-rmse:1.56719	val-rmse:1.58682
[5]	train-rmse:1.42645	val-rmse:1.44690
[6]	train-rmse:1.30047	val-rmse:1.32282
[7]	train-rmse:1.18786	val-rmse:1.21223
[8]	train-rmse:1.08744	val-rmse:1.11406
[9]	train-rmse:0.99801	val-rmse:1.02618
[10]	train-rmse:0.91846	val-rmse:0.94813
[11]	train-rmse:0.84797	val-rmse:0.87930
[12]	train-rmse:0.78540	val-rmse:0.81877
[13]	train-rmse:0.73026	val-rmse:0.76503
[14]	train-rmse:0.68164	val-rmse:0.71857
[15]	train-rmse:0.63889	val-rmse:0.67720
[16]	train-rmse:0.60130	val-rmse:0.64193
[17]	train-rmse:0.56852	val-rmse:0.61125
[18]	train-rmse:0.53982	val-rmse:0.58452
[19]	train-rmse:0.51488	val-rmse:0.56095
[20]	train-rmse:0.49316	val-rmse:0.54109
[21]	train-rmse:0.47428	val-rmse:0.52411
[22]	train-rmse:0.45775	val-rmse:0.50950
[23]	train-rmse:0.44362	val-rmse:0.49691
[24]	train-rmse:0.43128	va

In [ ]:
train-rmse:0.20896	val-rmse:0.45420